<a href="https://colab.research.google.com/github/viniciuspdasilva/pos-ia/blob/master/mini_projeto_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de Sentimentos com Redes Neurais

Este notebook foi adaptado do [github da Udacity para o Deep Learning Nanodegree](https://github.com/udacity/deep-learning-v2-pytorch) com a metodologia de Andrew Trask.

### Visão Geral do Tutorial

- [Analisando o Dataset](#lesson_1)
- [Desenvolvendo uma "Teoria de Predição"](#lesson_2)
- [**PROJETO 1**: Rápida Validação da Teoria](#project_1)


- [Transformando Texto em Números](#lesson_3)
- [**PROJETO 2**: Criando Dados de Entrada/Saída](#project_2)


- [**PROJETO 3**: Construindo Nossa Rede Neural](#project_3)


- [Entendendo Ruído da Rede Neural](#lesson_4)
- [**PROJETO 4**: Acelerando o Aprendizado através da Redução de Ruído](#project_4)


- [Mais Redução de Ruído](#lesson_5)
- [**PROJETO 5**: Reduzindo o Ruído através da Redução Estratégica do Vocabulário](#project_5)

# Analisando o Dataset<a id='lesson_1'></a>

In [2]:
!mkdir data
!wget -c https://github.com/agungsantoso/deep-learning-v2-pytorch/raw/master/sentiment-rnn/data/labels.txt
!wget -c https://github.com/agungsantoso/deep-learning-v2-pytorch/raw/master/sentiment-rnn/data/reviews.txt
!mv *.txt data/
def pretty_print_review_and_label(i):
    print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

g = open('data/reviews.txt','r')
reviews = list(map(lambda x:x[:-1], g.readlines()))
g.close()

g = open('data/labels.txt','r')
labels = list(map(lambda x:x[:-1].upper(), g.readlines()))
g.close()

--2020-06-30 13:44:33--  https://github.com/agungsantoso/deep-learning-v2-pytorch/raw/master/sentiment-rnn/data/labels.txt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/agungsantoso/deep-learning-v2-pytorch/master/sentiment-rnn/data/labels.txt [following]
--2020-06-30 13:44:34--  https://raw.githubusercontent.com/agungsantoso/deep-learning-v2-pytorch/master/sentiment-rnn/data/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt’

labels.txt          100%[===================>] 219.73K  --.-KB/s    in 0.05s   

2020-06-30 13:44:35 (3.99 MB/s) - 

**Nota:** Os dados em `reviews.txt` que estamos utilizando já passaram por algumas etapas de pré-processamento e contêm apenas caracteres em letras minúsculas. Isso deve ser feito para podermos lidar com palavras iguais escritas de formas diferentes, como `The`, `the` e `THE`, que são convertidas para `the`.

In [8]:
len(reviews)

25000

In [3]:
reviews[0]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   '

In [4]:
labels[0]

'POSITIVE'

# Desenvolvendo uma "Teoria de Predição"<a id='lesson_2'></a>

In [5]:
print("labels.txt \t : \t reviews.txt\n")
pretty_print_review_and_label(2137)
pretty_print_review_and_label(12816)
pretty_print_review_and_label(6267)
pretty_print_review_and_label(21934)
pretty_print_review_and_label(5297)
pretty_print_review_and_label(4998)

labels.txt 	 : 	 reviews.txt

NEGATIVE	:	this movie is terrible but it has some good effects .  ...
POSITIVE	:	adrian pasdar is excellent is this film . he makes a fascinating woman .  ...
NEGATIVE	:	comment this movie is impossible . is terrible  very improbable  bad interpretat...
POSITIVE	:	excellent episode movie ala pulp fiction .  days   suicides . it doesnt get more...
NEGATIVE	:	if you haven  t seen this  it  s terrible . it is pure trash . i saw this about ...
POSITIVE	:	this schiffer guy is a real genius  the movie is of excellent quality and both e...


# Projeto 1: Rápida Validação da Teoria<a id='project_1'></a>

Tenha em mente que a forma como resolveremos nossos problemas nesses projetos é apenas uma de várias possíveis.

Um link extra que pode vir a ser útil em seus estudos é a documentação da classe [Counter](https://docs.python.org/2/library/collections.html#collections.Counter).

In [7]:
from collections import Counter
import numpy as np

Iremos criar 3 objetos `Counter`: um para palavras de reviews positivas; uma para palavras de reviews negativas; e um para todas as palavras.

In [8]:
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()

Agora, iremos examinar todas as reviews. Para cada palavra em uma review positiva, iremos incrementar o contador para aquela palavra tanto no contador positivo quando no contador totarl. Faremos o mesmo processo para as palavras de reviews negativas.

**Nota:** Utilizaremos a função `split(' ')` para dividir as reviews em palavras individualmente.

In [9]:
for i in range(len(reviews)):
    if(labels[i] == 'POSITIVE'):
        for word in reviews[i].split(' '):
            positive_counts[word] += 1
            total_counts[word] += 1
    else:
        for word in reviews[i].split(' '):
            negative_counts[word] += 1
            total_counts[word] += 1

As duas células abaixo listam todas as palavras utilizadas nas reviews positivas e nas negativas, ordenadas das mais frequentes para as menos frequentes.

In [10]:
positive_counts.most_common()

[('', 550468),
 ('the', 173324),
 ('.', 159654),
 ('and', 89722),
 ('a', 83688),
 ('of', 76855),
 ('to', 66746),
 ('is', 57245),
 ('in', 50215),
 ('br', 49235),
 ('it', 48025),
 ('i', 40743),
 ('that', 35630),
 ('this', 35080),
 ('s', 33815),
 ('as', 26308),
 ('with', 23247),
 ('for', 22416),
 ('was', 21917),
 ('film', 20937),
 ('but', 20822),
 ('movie', 19074),
 ('his', 17227),
 ('on', 17008),
 ('you', 16681),
 ('he', 16282),
 ('are', 14807),
 ('not', 14272),
 ('t', 13720),
 ('one', 13655),
 ('have', 12587),
 ('be', 12416),
 ('by', 11997),
 ('all', 11942),
 ('who', 11464),
 ('an', 11294),
 ('at', 11234),
 ('from', 10767),
 ('her', 10474),
 ('they', 9895),
 ('has', 9186),
 ('so', 9154),
 ('like', 9038),
 ('about', 8313),
 ('very', 8305),
 ('out', 8134),
 ('there', 8057),
 ('she', 7779),
 ('what', 7737),
 ('or', 7732),
 ('good', 7720),
 ('more', 7521),
 ('when', 7456),
 ('some', 7441),
 ('if', 7285),
 ('just', 7152),
 ('can', 7001),
 ('story', 6780),
 ('time', 6515),
 ('my', 6488),
 ('g

In [11]:
negative_counts.most_common()

[('', 561462),
 ('.', 167538),
 ('the', 163389),
 ('a', 79321),
 ('and', 74385),
 ('of', 69009),
 ('to', 68974),
 ('br', 52637),
 ('is', 50083),
 ('it', 48327),
 ('i', 46880),
 ('in', 43753),
 ('this', 40920),
 ('that', 37615),
 ('s', 31546),
 ('was', 26291),
 ('movie', 24965),
 ('for', 21927),
 ('but', 21781),
 ('with', 20878),
 ('as', 20625),
 ('t', 20361),
 ('film', 19218),
 ('you', 17549),
 ('on', 17192),
 ('not', 16354),
 ('have', 15144),
 ('are', 14623),
 ('be', 14541),
 ('he', 13856),
 ('one', 13134),
 ('they', 13011),
 ('at', 12279),
 ('his', 12147),
 ('all', 12036),
 ('so', 11463),
 ('like', 11238),
 ('there', 10775),
 ('just', 10619),
 ('by', 10549),
 ('or', 10272),
 ('an', 10266),
 ('who', 9969),
 ('from', 9731),
 ('if', 9518),
 ('about', 9061),
 ('out', 8979),
 ('what', 8422),
 ('some', 8306),
 ('no', 8143),
 ('her', 7947),
 ('even', 7687),
 ('can', 7653),
 ('has', 7604),
 ('good', 7423),
 ('bad', 7401),
 ('would', 7036),
 ('up', 6970),
 ('only', 6781),
 ('more', 6730),
 ('

Como pudemos ver, palavras comuns como `the` aparecem muito frequentemente nas reviews positivas e nas negativas. Em vez que encontrar as palavras mais frequente em cada tipo de review, o que realmente queremos são as palavras encontradas nas reviews positivas que são mais comuns do que nas negativas (e vice versa). Para isso, iremos calcular as **frequências** com que cada palavra aparece em cada classe de review.

>Importante: a taxa positive-to-negative para uma dada palavra será calculada como `positive_counts[word] / float(negative_counts[word]+1)`. Note que o `+1` no denominador garante que não iremos realizar uma divisão por zero (caso em que uma palavra aparece apenas em reviews positivas).

Iremos realizar esse cálculo apenas para as palavras mais comuns (consideraremos como comuns apenas as palavras que apareceram pelo menos 100 vezes).

In [12]:
pos_neg_ratios = Counter()

for term,cnt in list(total_counts.most_common()):
    if(cnt > 100):
        pos_neg_ratio = positive_counts[term] / float(negative_counts[term]+1)
        pos_neg_ratios[term] = pos_neg_ratio

Vamos visualizar as frequências de algumas palavras:

In [13]:
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'terrible' = {}".format(pos_neg_ratios["terrible"]))

Pos-to-neg ratio for 'the' = 1.0607993145235326
Pos-to-neg ratio for 'amazing' = 4.022813688212928
Pos-to-neg ratio for 'terrible' = 0.17744252873563218


Vemos o seguinte:

* Palavras que a gente esperaria ver com mais frequência em críticas positivas - como "amazing" - têm uma proporção maior que 1. Quanto maior a relação dessa palavra com reviews positivas, mais distante de 1 será sua taxa.
* Palavras que a gente esperaria ver com mais frequência em críticas negativas - como "terrible" - têm taxas inferiores a 1. Quanto maior a relação dessa palavra com reviews negativas, mais próxima de zero será sua relação positivo / negativo.
* Palavras neutras, que realmente não transmitem nenhum sentimento, porque você esperaria vê-las em todos os tipos de críticas - como "the" - têm valores muito próximos de 1. Uma palavra perfeitamente neutra - usada exatamente na mesma frequência de críticas positivas como críticas negativas - seria quase exatamente 1. O `+ 1` que sugerimos que você adicione ao denominador enviesa levemente as palavras para negativas, mas isso não fará diferença porque será um pequeno viés e mais tarde estaremos ignorando palavras que são muito próximas de neutras.

Ok, as frequências nos dizem quais palavras são usadas com mais frequência em reviews positivas ou negativas, mas os valores específicos que calculamos são um pouco difíceis de trabalhar. Uma palavra muito positiva como "amazing" tem um valor acima de 4, enquanto uma palavra muito negativa como "terrible" tem um valor em torno de 0,18. Esses valores não são fáceis de comparar por alguns motivos:

* No momento, 1 é considerado neutro, mas o valor absoluto das taxas postive-to-negative de palavras muito positivas é maior que o valor absoluto das taxas para as palavras muito negativas. Entretanto, não há como comparar diretamente dois números e ver se uma palavra transmite a mesma magnitude de sentimento positivo que outra palavra transmite sentimento negativo. Portanto, devemos centralizar todos os valores ao redor do valor neutro. Assim, poderemos comparar o quanto de positividade ou de negatividade uma palavra transmite.
* Ao comparar valores absolutos, é mais fácil fazer isso em torno de zero do que de 1.

Para corrigir esses problemas, converteremos todas as nossas taxas para novos valores utilizando logaritmos (`np.log(ratio)`).

No final, palavras extremamente positivas e extremamente negativas terão proporções positivas-negativas com magnitudes semelhantes, mas sinais opostos.

In [16]:
for word, ratio in pos_neg_ratios.most_common():
    pos_neg_ratios[word] = np.log(ratio)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


Vamos dar uma olhada nos novos valores para as mesmas palavras que vimos anteriormente:

In [15]:
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'terrible' = {}".format(pos_neg_ratios["terrible"]))

Pos-to-neg ratio for 'the' = 0.05902269426102881
Pos-to-neg ratio for 'amazing' = 1.3919815802404802
Pos-to-neg ratio for 'terrible' = -1.7291085042663878


Agora, vemos que palavras neutras possuem taxas próximas de zero (`the`). A palavra `amazing` possui um valor bem maior do que zero, indicando sua clara significância positiva. `terrible` possui um valor bem baixo, também indicando que ela tem um forte sentido negativo. Note, entretanto, que `terrible` é uma palavra mais relacionada a reviews negativas do que `amazing` se relaciona com reviews positivas.

Vamos, agora, ver as taxas das demais palavras. A próxima célula exibe todas as palavras, ordenadas de acordo com sua relação com reviews positivas. A célula seguinte, por sua vez, exibe as 30 palavras mais relacionadas com reviews negativas.

Relembrando: palavras neutras terão valores próximos de `0`. Palavras positivas terão valores maiores (até maiores do que `1`). Palavras relacionadas a reviews negativas terão valores negativos (até menores do que `-1`). Esses novos valores são a razão para termos utilizado o logaritmo.

In [17]:
pos_neg_ratios.most_common()

[('', nan),
 ('loved', 0.14528245890406116),
 ('the', -2.8298332605426704),
 ('.', nan),
 ('and', -1.674252477599604),
 ('to', nan),
 ('br', nan),
 ('it', nan),
 ('i', nan),
 ('this', nan),
 ('that', nan),
 ('was', nan),
 ('as', -1.4133751189249417),
 ('is', -2.0126153400118763),
 ('of', -2.2286925693498265),
 ('with', -2.2308951204645417),
 ('a', -2.92657791456808),
 ('movie', nan),
 ('but', nan),
 ('film', -2.4578532477038557),
 ('you', nan),
 ('on', nan),
 ('t', nan),
 ('not', nan),
 ('his', -1.0517858419568342),
 ('he', -1.8246772654209993),
 ('in', -1.9824545891751846),
 ('s', -2.6674939322585116),
 ('for', -3.816229733505857),
 ('are', -4.387157598400469),
 ('have', nan),
 ('be', nan),
 ('one', -3.2486823642488063),
 ('all', nan),
 ('at', nan),
 ('they', nan),
 ('who', -1.9687512871713466),
 ('by', -2.0515870366701274),
 ('an', -2.350339992429906),
 ('so', nan),
 ('from', -2.2919768012950548),
 ('like', nan),
 ('there', nan),
 ('her', -1.2874425650932688),
 ('or', nan),
 ('just',

In [18]:
list(reversed(pos_neg_ratios.most_common()))[0:30]

[('hartley', 0.47588499532711054),
 ('melting', nan),
 ('niro', nan),
 ('angeles', -3.238550274970754),
 ('specifically', -1.85032811106515),
 ('cinematographer', -1.2864368031365205),
 ('cure', nan),
 ('amanda', -2.1389110278431644),
 ('authority', -1.15094614041988),
 ('emphasis', -1.15094614041988),
 ('blew', -1.030930433158723),
 ('eighties', -1.030930433158723),
 ('oz', -0.8249545038324405),
 ('planned', -0.7348589869664729),
 ('ruthless', -0.4317917973011473),
 ('matches', -0.2449299875154264),
 ('adorable', 0.0701199182449436),
 ('soccer', 0.5640959754919254),
 ('senseless', nan),
 ('disappear', nan),
 ('practice', nan),
 ('advance', nan),
 ('outfit', nan),
 ('peoples', -0.5006512197182454),
 ('saga', -0.4317917973011473),
 ('backgrounds', -0.2449299875154264),
 ('awe', -0.02854291790698025),
 ('diana', 0.0701199182449436),
 ('april', 0.30005030723545273),
 ('simplistic', nan)]

# Fim do Projeto 1.

# Transformando Texto em Números<a id='lesson_3'></a>

In [ ]:
from IPython.display import Image

review = "This was a horrible, terrible movie."

Image(filename='sentiment_network.png')

FileNotFoundError: ignored

In [ ]:
review = "The movie was excellent"

Image(filename='sentiment_network_pos.png')

# Projeto 2: Criando Dados de Entrada/Saída<a id='project_2'></a>

Vamos crias um [set](https://docs.python.org/3/tutorial/datastructures.html#sets) chamado `vocab` que contém todas as palavras do vocabulário.

In [20]:
vocab = set(total_counts.keys())

Vamos ver o tamanho do vocabulário (deveria haver **74074** palavras).

In [21]:
vocab_size = len(vocab)
print(vocab_size)

74074


A imagem abaixo representa as camadas de uma rede neural, que iremos construir mais adiante. `layer_0` é a camada de entrada, `layer_1` é uma camada oculta e `layer_2` é a camada de saída.

In [ ]:
from IPython.display import Image
Image(filename='sentiment_network_2.png')

Vamos criar um array numpy chamado `layer_0` e inicializá-lo com zeros. Esse array possuirá 1 linha e `vocab_size` colunas. 

In [22]:
layer_0 = np.zeros((1, vocab_size))

A célula abaixo deve resulta em `(1, 74074)`

In [23]:
layer_0.shape

(1, 74074)

In [ ]:
from IPython.display import Image
Image(filename='sentiment_network.png')

`layer_0` possui uma entrada para cada palavra do vocabulário. Precisamos saber o índice de cada palavra, então iremos executar a célula abaixo para criar uma lookup table que armazena o índice de cada palavra.

In [24]:
word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i

word2index

{'': 0,
 'sacchi': 1,
 'switchblade': 2,
 'cels': 3,
 'reprises': 4,
 'dilemma': 5,
 'surfers': 6,
 'arrrrrggghhhhhh': 7,
 'lenghts': 8,
 'disappoint': 9,
 'inveigh': 10,
 'lenge': 11,
 'chuckling': 12,
 'huzzah': 13,
 'prequels': 14,
 'undated': 15,
 'aesthetics': 16,
 'doodads': 17,
 'sugary': 18,
 'upstanding': 19,
 'punt': 20,
 'feinting': 21,
 'qv': 22,
 'minium': 23,
 'sherlock': 24,
 'headdress': 25,
 'useless': 26,
 'cheesefest': 27,
 'belmore': 28,
 'blythe': 29,
 'remand': 30,
 'comme': 31,
 'deceptiveness': 32,
 'resurrection': 33,
 'ago': 34,
 'statute': 35,
 'scapegoat': 36,
 'polluted': 37,
 'descent': 38,
 'hippies': 39,
 'allegation': 40,
 'forcibly': 41,
 'potter': 42,
 'dos': 43,
 'text': 44,
 'broadens': 45,
 'shabbiness': 46,
 'femi': 47,
 'pinnacles': 48,
 'menjou': 49,
 'wnk': 50,
 'galleons': 51,
 'shallowly': 52,
 'rishi': 53,
 'venessa': 54,
 'cambodian': 55,
 'mastodon': 56,
 'pretagonist': 57,
 'sunlight': 58,
 'shunned': 59,
 'hightlight': 60,
 'rousset': 61


`update_input_layer` é uma função que conta quantas vezes cada palavra é usada em uma review e guarda essa contagem nos índices apropriados dentro de `layer_0`.

In [25]:
def update_input_layer(review):
    global layer_0
    
    # limpamos o estado anterior, resetando tudo para 0.
    layer_0 *= 0
    
    # contamos quantas vezes cada palavra é utilizada na review e armazena os resultados em layer_0 
    for word in review.split(' '):
        layer_0[0][word2index[word]] += 1

A próxima célula atualiza a camada de entrada com a primeira review.

In [27]:
update_input_layer(reviews[6])
layer_0

array([[15.,  0.,  0., ...,  0.,  0.,  0.]])

`get_target_for_labels` retorna `0` ou `1`, para reviews `NEGATIVE` ou `POSITIVE`, respectivamente.

In [28]:
def get_target_for_label(label):
    if(label == 'POSITIVE'):
        return 1
    else:
        return 0

In [29]:
labels[0]

'POSITIVE'

In [30]:
get_target_for_label(labels[0])

1

In [31]:
labels[1]

'NEGATIVE'

In [32]:
get_target_for_label(labels[1])

0

# Fim do Projeto 2.

# Projeto 3: Construindo Nossa Rede Neural<a id='project_3'></a>

Iremos construir uma rede neural sem o uso de qualquer framework de ML. Iremos implementar tudo na mão.

Entretanto, não se preocupe aqui. O objetivo não é que você entenda a implementação de uma rede neural, apenas que possamos a utilizar. Você terá a oportunidade de montar essa rede utilizando PyTorch mais para frente.

In [ ]:
import time
import sys
import numpy as np

class SentimentNetwork:
    def __init__(self, reviews, labels, hidden_nodes=10, learning_rate=0.1):
        # para que possamos ter sempre os mesmos resultados, iremos manter
        # a inicialização aleatória como sendo fixa
        np.random.seed(1)

        self.pre_process_data(reviews, labels)
        
        self.init_network(len(self.review_vocab),hidden_nodes, 1, learning_rate)

    def pre_process_data(self, reviews, labels):
        # populando review_vocab com as palavras da review
        review_vocab = set()
        for review in reviews:
            for word in review.split(' '):
                review_vocab.add(word)

        # convertendo para uma lista para que possamos acessar as palavras por seus índices
        self.review_vocab = list(review_vocab)
        
        # populando label_vocab com todas as palavras nas labels
        label_vocab = set()
        for label in labels:
            label_vocab.add(label)
        
        # convertendo em lista para acessar por índice
        self.label_vocab = list(label_vocab)
        
        self.review_vocab_size = len(self.review_vocab)
        self.label_vocab_size = len(self.label_vocab)
        
        # criando um dicionário das palavras no vocabulário mapeadas para índices
        self.word2index = {}
        for i, word in enumerate(self.review_vocab):
            self.word2index[word] = i
        
        # criando um dicionário de labels mapeadas em índices
        self.label2index = {}
        for i, label in enumerate(self.label_vocab):
            self.label2index[label] = i
        
    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        self.learning_rate = learning_rate

        self.weights_0_1 = np.zeros((self.input_nodes,self.hidden_nodes))
    
        self.weights_1_2 = np.random.normal(0.0, self.hidden_nodes**-0.5, 
                                                (self.hidden_nodes, self.output_nodes))
        
        self.layer_0 = np.zeros((1,input_nodes))
    
    def update_input_layer(self,review):
        self.layer_0 *= 0
        
        for word in review.split(" "):
            if(word in self.word2index.keys()):
                self.layer_0[0][self.word2index[word]] += 1
                
    def get_target_for_label(self,label):
        if(label == 'POSITIVE'):
            return 1
        else:
            return 0
        
    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_output_2_derivative(self,output):
        return output * (1 - output)
    
    def train(self, training_reviews, training_labels):
        assert(len(training_reviews) == len(training_labels))
        
        correct_so_far = 0

        start = time.time()
        
        for i in range(len(training_reviews)):
            review = training_reviews[i]
            label = training_labels[i]

            ### Forward pass ###

            # Input Layer
            self.update_input_layer(review)

            # Hidden layer
            layer_1 = self.layer_0.dot(self.weights_0_1)

            # Output layer
            layer_2 = self.sigmoid(layer_1.dot(self.weights_1_2))
            
            ### Backward pass ###

            # Output error
            layer_2_error = layer_2 - self.get_target_for_label(label) # Output layer error is the difference between desired target and actual output.
            layer_2_delta = layer_2_error * self.sigmoid_output_2_derivative(layer_2)

            # Backpropagated error
            layer_1_error = layer_2_delta.dot(self.weights_1_2.T) # errors propagated to the hidden layer
            layer_1_delta = layer_1_error # hidden layer gradients - no nonlinearity so it's the same as the error

            # Update the weights
            self.weights_1_2 -= layer_1.T.dot(layer_2_delta) * self.learning_rate # update hidden-to-output weights with gradient descent step
            self.weights_0_1 -= self.layer_0.T.dot(layer_1_delta) * self.learning_rate # update input-to-hidden weights with gradient descent step

            # Keep track of correct predictions.
            if(layer_2 >= 0.5 and label == 'POSITIVE'):
                correct_so_far += 1
            elif(layer_2 < 0.5 and label == 'NEGATIVE'):
                correct_so_far += 1
            
            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) \
                             + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4] + "%")
            if(i % 2500 == 0):
                print("")
    
    def test(self, testing_reviews, testing_labels):
        # keep track of how many correct predictions we make
        correct = 0

        # we'll time how many predictions per second we make
        start = time.time()

        # Loop through each of the given reviews and call run to predict
        # its label. 
        for i in range(len(testing_reviews)):
            pred = self.run(testing_reviews[i])
            if(pred == testing_labels[i]):
                correct += 1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the prediction process. 

            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
    
    def run(self, review):
        # Run a forward pass through the network, like in the "train" function.
        
        # Input Layer
        self.update_input_layer(review.lower())

        # Hidden layer
        layer_1 = self.layer_0.dot(self.weights_0_1)

        # Output layer
        layer_2 = self.sigmoid(layer_1.dot(self.weights_1_2))
        
        # Return POSITIVE for values above greater-than-or-equal-to 0.5 in the output layer;
        # return NEGATIVE for other values
        if(layer_2[0] >= 0.5):
            return "POSITIVE"
        else:
            return "NEGATIVE"
        

A célula abaixo irá criar um objeto do tipo `SentimentNetwork` que irá treinar sobre todos os dados que possuímos, exceto pelas últimas 1000 reviews (que serão utilizadas para teste).

In [ ]:
mlp = SentimentNetwork(reviews[:-1000],labels[:-1000], learning_rate=0.1)

A célula abaixo irá testar a rede sobre o conjunto de testes nas últimas 1000 reviews que separamos acima.

**Ainda não treinamos a rede, então os resultados provavelmente serão em torno de 50%.**

In [ ]:
mlp.test(reviews[-1000:],labels[-1000:])

A célula abaixo realiza o treinamento da rede. Iremos exibir também a acurácia para que possamos ver a evolução.

In [ ]:
mlp.train(reviews[:-1000], labels[:-1000])

Observe que o treinamento não foi bom. Parte da razão pode ser por conta da taxa de aprendizado que está muito alta.

A célula abaixo irá refazer o treinamento com uma taxa menor (de `0.01`). Vamos ver se melhoramos nossos resultados.

In [ ]:
mlp = SentimentNetwork(reviews[:-1000], labels[:-1000], learning_rate=0.01)
mlp.train(reviews[:-1000],labels[:-1000])

### Diferentes Inicializações, Diferentes Resultados

Com uma inicialização um pouco melhor (`hidden_nodes**-0.5` em vez de `output_nodes**-0.5`), nós realmente vemos uma melhoria com um learning rate de 0.01. Essa solução não é perfeita, mas claramente mostra potencial e iremos melhorar isso daqui a pouco.

Porque essa estratégia de inicialização é melhor? Ainda vamos chegar lá, mas é suficiente dizer que as melhores inicializações de pesos são uma função de $1/\sqrt(n)$, onde n é o número de nós naquela camada. Sendo assim, os pesos entre a camada oculta e a camada de saída devem ser uma função do tamanho da camada escondida. Assim, utilizar `hidden_nodes**-0.5` em vez de `output_nodes**-0.5` é uma melhor estratégia de inicialização.

# Fim do Projeto 3.

# Entendendo Ruído da Rede Neural<a id='lesson_4'></a>

In [ ]:
from IPython.display import Image
Image(filename='sentiment_network.png')

In [ ]:
def update_input_layer(review):
    global layer_0
    
    layer_0 *= 0
    for word in review.split(" "):
        layer_0[0][word2index[word]] += 1

update_input_layer(reviews[0])

In [ ]:
layer_0

In [ ]:
review_counter = Counter()

In [ ]:
for word in reviews[0].split(" "):
    review_counter[word] += 1

In [ ]:
review_counter.most_common()

# Projeto 4: Reduzindo Ruído nos Dados de Entrada<a id='project_4'></a>

Faremos o seguinte para reduzir o ruído nos dados de entrada:
* Copiaremos a classe `SentimentNetwork` que fizemos anteriormente.
* Modificaremos o método `update_input_layer` para que não seja contada quantas vezes cada palavra é usada, mas, em vez disso, indicaremos apenas se a palavra foi utilizada ou não.

In [ ]:
import time
import sys
import numpy as np

class SentimentNetwork:
    def __init__(self, reviews,labels,hidden_nodes = 10, learning_rate = 0.1):
        np.random.seed(1)

        self.pre_process_data(reviews, labels)
        
        self.init_network(len(self.review_vocab), hidden_nodes, 1, learning_rate)

    def pre_process_data(self, reviews, labels):
        review_vocab = set()
        for review in reviews:
            for word in review.split(" "):
                review_vocab.add(word)

        self.review_vocab = list(review_vocab)
        
        label_vocab = set()
        for label in labels:
            label_vocab.add(label)
        
        self.label_vocab = list(label_vocab)
        
        self.review_vocab_size = len(self.review_vocab)
        self.label_vocab_size = len(self.label_vocab)
        
        self.word2index = {}
        for i, word in enumerate(self.review_vocab):
            self.word2index[word] = i
        
        self.label2index = {}
        for i, label in enumerate(self.label_vocab):
            self.label2index[label] = i
        
    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        self.learning_rate = learning_rate

        self.weights_0_1 = np.zeros((self.input_nodes,self.hidden_nodes))
    
        self.weights_1_2 = np.random.normal(0.0, self.hidden_nodes**-0.5, 
                                                (self.hidden_nodes, self.output_nodes))
        
        self.layer_0 = np.zeros((1,input_nodes))
    
        
    def update_input_layer(self,review):
        self.layer_0 *= 0
        
        for word in review.split(' '):
            if(word in self.word2index.keys()):
                ## ALTERADO PARA ESTE PROJETO: em vez de somar 1, setamos para 1
                self.layer_0[0][self.word2index[word]] = 1
                
    def get_target_for_label(self,label):
        if(label == 'POSITIVE'):
            return 1
        else:
            return 0
        
    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_output_2_derivative(self,output):
        return output * (1 - output)
    
    def train(self, training_reviews, training_labels):
        assert(len(training_reviews) == len(training_labels))
        
        correct_so_far = 0

        start = time.time()
        
        for i in range(len(training_reviews)):
            review = training_reviews[i]
            label = training_labels[i]
            
            ### Forward pass ###

            # Input Layer
            self.update_input_layer(review)

            # Hidden layer
            layer_1 = self.layer_0.dot(self.weights_0_1)

            # Output layer
            layer_2 = self.sigmoid(layer_1.dot(self.weights_1_2))
            
            ### Backward pass ###

            # Output error
            layer_2_error = layer_2 - self.get_target_for_label(label) # Output layer error is the difference between desired target and actual output.
            layer_2_delta = layer_2_error * self.sigmoid_output_2_derivative(layer_2)

            # Backpropagated error
            layer_1_error = layer_2_delta.dot(self.weights_1_2.T) # errors propagated to the hidden layer
            layer_1_delta = layer_1_error # hidden layer gradients - no nonlinearity so it's the same as the error

            # Update the weights
            self.weights_1_2 -= layer_1.T.dot(layer_2_delta) * self.learning_rate # update hidden-to-output weights with gradient descent step
            self.weights_0_1 -= self.layer_0.T.dot(layer_1_delta) * self.learning_rate # update input-to-hidden weights with gradient descent step

            # Keep track of correct predictions.
            if(layer_2 >= 0.5 and label == 'POSITIVE'):
                correct_so_far += 1
            elif(layer_2 < 0.5 and label == 'NEGATIVE'):
                correct_so_far += 1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the training process. 
            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) \
                             + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4] + "%")
            if(i % 2500 == 0):
                print("")
    
    def test(self, testing_reviews, testing_labels):
        # keep track of how many correct predictions we make
        correct = 0

        # we'll time how many predictions per second we make
        start = time.time()

        # Loop through each of the given reviews and call run to predict
        # its label. 
        for i in range(len(testing_reviews)):
            pred = self.run(testing_reviews[i])
            if(pred == testing_labels[i]):
                correct += 1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the prediction process. 

            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
    
    def run(self, review):
        """
        Returns a POSITIVE or NEGATIVE prediction for the given review.
        """
        # Run a forward pass through the network, like in the "train" function.
        
        # Input Layer
        self.update_input_layer(review.lower())

        # Hidden layer
        layer_1 = self.layer_0.dot(self.weights_0_1)

        # Output layer
        layer_2 = self.sigmoid(layer_1.dot(self.weights_1_2))
        
        # Return POSITIVE for values above greater-than-or-equal-to 0.5 in the output layer;
        # return NEGATIVE for other values
        if(layer_2[0] >= 0.5):
            return "POSITIVE"
        else:
            return "NEGATIVE"
        

A célula abaixo recria a rede e a treina. Note que retornamos ao learning rate de `0.1`.

In [ ]:
mlp = SentimentNetwork(reviews[:-1000], labels[:-1000], learning_rate=0.1)
mlp.train(reviews[:-1000], labels[:-1000])

In [ ]:
mlp.test(reviews[-1000:], labels[-1000:])

# Fim do Projeto 4.

# Analisando o Vocabulário

In [ ]:
pos_neg_ratios.most_common()

In [ ]:
list(reversed(pos_neg_ratios.most_common()))[0:30]

In [ ]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Agora, vamos dar uma olhada na distribuição das palavras e suas afinidades com reviews positivas e negativas.

In [ ]:
hist, edges = np.histogram(list(map(lambda x:x[1],pos_neg_ratios.most_common())), density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Word Positive/Negative Affinity Distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

Ainda, vamos dar uma olhada na distribuição da frequência com que as palavras aparecem.

In [ ]:
frequency_frequency = Counter()

for word, cnt in total_counts.most_common():
    frequency_frequency[cnt] += 1

In [ ]:
hist, edges = np.histogram(list(map(lambda x:x[1],frequency_frequency.most_common())), density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="The frequency distribution of the words in our corpus")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

# Projeto 5: Reduzindo o Ruído através da Redução Estratégica do Vocabulário<a id='project_5'></a>

Aqui, iremos reduzir o ruído dos nossos dados através da redução do nosso vocabulário. Faremos o seguinte (a partir da mesma classe `SentimentNetwork`):

* Modificaremos o método `pre_process_data`:
>* Adicionaremos dois parâmetros adicionais: `min_count` e `polarity_cutoff`.
>* Calcularemos a taxa positive-to-negative das palavras da review.
>* Adicionaremos ao vocabulário apenas palavras que ocorrem mais do que `min_count` vezes.
>* Adicionaremos ao vocabulário apenas palavras cujo valor da taxa postive-to-negative é pelo menos `polarity_cutoff`.
* Modificaremos o método `__init__`:
>* Adicionaremos os mesmos dois parâmetros (`min_count` e `polarity_cutoff`) e utilizá-los na chamada ao método `pre_process_data`.

Procure pelos pontos onde há anotações de alterações de código:

In [ ]:
import time
import sys
import numpy as np

class SentimentNetwork:
    ## ALTERAÇÃO REALIZADA NESTE PROJETO: adição de min_count e polarity_cutoff
    def __init__(self, reviews, labels, min_count=10, polarity_cutoff=0.1, hidden_nodes=10, learning_rate=0.1):
        np.random.seed(1)

        ## ALTERAÇÃO REALIZADA NESTE PROJETO: adição dos parâmetros min_count e polarity_cutoff
        self.pre_process_data(reviews, labels, polarity_cutoff, min_count)
        self.init_network(len(self.review_vocab),hidden_nodes, 1, learning_rate)

    ## ALTERAÇÃO REALIZADA NESTE PROJETO: adição dos parâmetros min_count e polarity_cutoff
    def pre_process_data(self, reviews, labels, polarity_cutoff, min_count):
        
        ## ----------------------------------------
        ## ALTERAÇÃO REALIZADA NESTE PROJETO: cálculo da taxa positive-to-negative para as palavras antes
        # da construção do vocabulário
        positive_counts = Counter()
        negative_counts = Counter()
        total_counts = Counter()

        for i in range(len(reviews)):
            if(labels[i] == 'POSITIVE'):
                for word in reviews[i].split(" "):
                    positive_counts[word] += 1
                    total_counts[word] += 1
            else:
                for word in reviews[i].split(" "):
                    negative_counts[word] += 1
                    total_counts[word] += 1

        pos_neg_ratios = Counter()

        for term,cnt in list(total_counts.most_common()):
            if(cnt >= 50):
                pos_neg_ratio = positive_counts[term] / float(negative_counts[term]+1)
                pos_neg_ratios[term] = pos_neg_ratio

        for word,ratio in pos_neg_ratios.most_common():
            if(ratio > 1):
                pos_neg_ratios[word] = np.log(ratio)
            else:
                pos_neg_ratios[word] = -np.log((1 / (ratio + 0.01)))
        #
        ## ----------------------------------------
        
        review_vocab = set()
        for review in reviews:
            for word in review.split(" "):
                ## ALTERAÇÃO REALIZADA NESTE PROJETO: apenas iremos adicionar palavras que ocorrem
                # pelo menos min_count vezes e que possuem uma taxa de pelo menos polarity_cutoff
                if(total_counts[word] > min_count):
                    if(word in pos_neg_ratios.keys()):
                        if((pos_neg_ratios[word] >= polarity_cutoff) or (pos_neg_ratios[word] <= -polarity_cutoff)):
                            review_vocab.add(word)
                    else:
                        review_vocab.add(word)
                        
        self.review_vocab = list(review_vocab)
        
        label_vocab = set()
        for label in labels:
            label_vocab.add(label)
        
        self.label_vocab = list(label_vocab)
        
        self.review_vocab_size = len(self.review_vocab)
        self.label_vocab_size = len(self.label_vocab)
        
        self.word2index = {}
        for i, word in enumerate(self.review_vocab):
            self.word2index[word] = i
        
        self.label2index = {}
        for i, label in enumerate(self.label_vocab):
            self.label2index[label] = i

    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        self.learning_rate = learning_rate

        self.weights_0_1 = np.zeros((self.input_nodes,self.hidden_nodes))

        self.weights_1_2 = np.random.normal(0.0, self.hidden_nodes**-0.5, 
                                                (self.hidden_nodes, self.output_nodes))
        
        ## ALTERAÇÃO REALIZADA NESTE PROJETO: removemos layer_0, adicionamos layer_1
        self.layer_1 = np.zeros((1,hidden_nodes))
    
    ## ALTERAÇÃO REALIZADA NESTE PROJETO: removemos o método update_input_layer
    
    def get_target_for_label(self,label):
        if(label == 'POSITIVE'):
            return 1
        else:
            return 0
        
    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_output_2_derivative(self,output):
        return output * (1 - output)
    
    def train(self, training_reviews_raw, training_labels):

        ## ALTERAÇÃO REALIZADA NESTE PROJETO: pré-processamos training_reviews
        # para que possamos lidar diretamente com os índices de entradas não-nulas.
        training_reviews = list()
        for review in training_reviews_raw:
            indices = set()
            for word in review.split(' '):
                if(word in self.word2index.keys()):
                    indices.add(self.word2index[word])
            training_reviews.append(list(indices))

        assert(len(training_reviews) == len(training_labels))
        
        correct_so_far = 0

        start = time.time()
        
        for i in range(len(training_reviews)):
            review = training_reviews[i]
            label = training_labels[i]
            ### Forward pass ###

            ## ALTERAÇÃO REALIZADA NESTE PROJETO: removemos a chamada para 'update_input_layer',
            # pois 'layer_0' não é mais utilizada.

            # Hidden layer
            ## ALTERAÇÃO REALIZADA NESTE PROJETO: adição dos pesos apenas para itens não-nulos
            self.layer_1 *= 0
            for index in review:
                self.layer_1 += self.weights_0_1[index]

            # Output layer
            ## ALTERAÇÃO REALIZADA NESTE PROJETO: utilização de 'self.layer_1' em vez de 'local layer_1'
            layer_2 = self.sigmoid(self.layer_1.dot(self.weights_1_2))            
            
            ### Backward pass ###

            # Output error
            layer_2_error = layer_2 - self.get_target_for_label(label) # Output layer error is the difference between desired target and actual output.
            layer_2_delta = layer_2_error * self.sigmoid_output_2_derivative(layer_2)

            # Backpropagated error
            layer_1_error = layer_2_delta.dot(self.weights_1_2.T) # errors propagated to the hidden layer
            layer_1_delta = layer_1_error # hidden layer gradients - no nonlinearity so it's the same as the error

            # Update the weights
            ## ALTERAÇÃO REALIZADA NESTE PROJETO: utilização de 'self.layer_1' em vez de 'layer_1'
            self.weights_1_2 -= self.layer_1.T.dot(layer_2_delta) * self.learning_rate # update hidden-to-output weights with gradient descent step
            
            ## ALTERAÇÃO REALIZADA NESTE PROJETO: atualização apenas dos pesos que foram utilizados na passada forward
            for index in review:
                self.weights_0_1[index] -= layer_1_delta[0] * self.learning_rate # update input-to-hidden weights with gradient descent step

            # Keep track of correct predictions.
            if(layer_2 >= 0.5 and label == 'POSITIVE'):
                correct_so_far += 1
            elif(layer_2 < 0.5 and label == 'NEGATIVE'):
                correct_so_far += 1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the training process. 
            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) \
                             + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4] + "%")
            if(i % 2500 == 0):
                print("")
    
    def test(self, testing_reviews, testing_labels):
        # keep track of how many correct predictions we make
        correct = 0

        # we'll time how many predictions per second we make
        start = time.time()

        # Loop through each of the given reviews and call run to predict
        # its label. 
        for i in range(len(testing_reviews)):
            pred = self.run(testing_reviews[i])
            if(pred == testing_labels[i]):
                correct += 1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the prediction process. 

            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
    
    def run(self, review):
        # Run a forward pass through the network, like in the "train" function.
        
        ## ALTERAÇÃO REALIZADA NESTE PROJETO: remoção da chamada ao método update_input_layer
        # pois layer_0 não é mais utilizada

        # Hidden layer
        ## ALTERAÇÃO REALIZADA NESTE PROJETO: identificação dos índices utilizados na review e
        # depois adicionamos apenas esses pesos ao layer_1
        self.layer_1 *= 0
        unique_indices = set()
        for word in review.lower().split(" "):
            if word in self.word2index.keys():
                unique_indices.add(self.word2index[word])
        for index in unique_indices:
            self.layer_1 += self.weights_0_1[index]
        
        # Output layer
        ## ALTERAÇÃO REALIZADA NESTE PROJETO: alteração para utilizar self.layer_1 em vez de layer_1
        layer_2 = self.sigmoid(self.layer_1.dot(self.weights_1_2))
         
        # Return POSITIVE for values above greater-than-or-equal-to 0.5 in the output layer;
        # return NEGATIVE for other values
        if(layer_2[0] >= 0.5):
            return "POSITIVE"
        else:
            return "NEGATIVE"


Vamos treinar a rede com um pequeno `polarity_cutoff`.

In [ ]:
mlp = SentimentNetwork(reviews[:-1000], labels[:-1000], min_count=20, polarity_cutoff=0.05, learning_rate=0.01)
mlp.train(reviews[:-1000], labels[:-1000])

Vamos testar sobre o conjunto de testes.

In [ ]:
mlp.test(reviews[-1000:], labels[-1000:])

Vamos aumentar o valor de `polarity_cutoff`.

In [ ]:
mlp = SentimentNetwork(reviews[:-1000], labels[:-1000], min_count=20, polarity_cutoff=0.8, learning_rate=0.01)
mlp.train(reviews[:-1000], labels[:-1000])

Vamos testar o conjunto de testes.

In [ ]:
mlp.test(reviews[-1000:], labels[-1000:])

# Fim do Projeto 5.

# Mini-projeto

Agora, que tal a gente entender melhor o que foi feito aqui por meio de uma prática?

Implemente essa mesma análise de sentimentos utilizando o PyTorch e obtenha resultados melhores do que os que obtivemos aqui (acima de 85% de acurácia). Faça a matriz de confusão e calcule as métricas que julgar interessantes. Implemente também um conjunto de validação e treine implementando o early stopping.

* Desafio: o que fizemos aqui é o básico do básico de NLP. Sendo assim, vá além! Utilize quaisquer métodos de classificação e de NLP para obter a maior taxa de acerto da turma neste dataset. O grupo que obtiver a melhor acurácia receberá 1 ponto extra neste projeto.

Utilize sempre as 1000 últimas reviews como sendo o conjunto de teste. Para os conjuntos de treinamento e validação, utilize os dados restantes.

In [30]:
from string import punctuation
from collections import Counter

import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader


In [32]:

with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
with open('data/labels.txt', 'r') as f:
    labels = f.read()

In [33]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [35]:

from string import punctuation

# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

# split by new lines and spaces
reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

# create a list of words
words = all_text.split()

words[:30]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me']

In [37]:
## Build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

In [38]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_ints[:1])

Unique words:  74072

Tokenized review: 
 [[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]]


In [39]:
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])

In [41]:
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


In [42]:
print('Number of reviews before removing outliers: ', len(reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.

# get indices of any reviews with length 0
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]

# remove 0-length reviews and their labels
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_ints))

Number of reviews before removing outliers:  25001
Number of reviews after removing outliers:  25000


In [43]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [44]:
seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

print(features[:30,:10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0

In [45]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*0.8)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


In [46]:
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[    0,     0,     0,  ...,     1,   574,  1860],
        [    0,     0,     0,  ...,    16,     3,   494],
        [ 1052,     1,  4296,  ...,    80,     5,     1],
        ...,
        [32123,  3871,     6,  ...,     1,  1284,    30],
        [    0,     0,     0,  ...,    32,     1,   130],
        [    0,     0,     0,  ...,  1094,     4,   612]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
        1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0,
        1, 0])


In [52]:
class SentimentalAnalytics(nn.Module):

  def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):

    super(SentimentalAnalytics, self).__init__()

    self.output_size = output_size
    self.n_layers = n_layers
    self.hidden_dim = hidden_dim

    self.embedding = nn.Embedding(vocab_size, embedding_dim=embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
    
         # dropout layer
    self.dropout = nn.Dropout(0.3)

    # linear and sigmoid layers
    self.fc = nn.Linear(hidden_dim, output_size)
    self.sig = nn.Sigmoid()
  def forward(self, x, hidden):
      """
      Perform a forward pass of our model on some input and hidden state.
      """
      batch_size = x.size(0)

      # embeddings and lstm_out
      embeds = self.embedding(x)
      lstm_out, hidden = self.lstm(embeds, hidden)
  
      # stack up lstm outputs
      lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
      
      # dropout and fully-connected layer
      out = self.dropout(lstm_out)
      out = self.fc(out)
      # sigmoid function
      sig_out = self.sig(out)
      
      # reshape to be batch_size first
      sig_out = sig_out.view(batch_size, -1)
      sig_out = sig_out[:, -1] # get last batch of labels
      
      # return last sigmoid output and hidden state
      return sig_out, hidden
    
  def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden 

In [54]:
vocab_leng = len(vocab_to_int)+1

output_size = 1

embedding_dim = 400

hidden_dim = 256

n_layers = 2

model = SentimentalAnalytics(vocab_leng, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.6)

print(model)

SentimentalAnalytics(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.6)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [55]:
lr = 0.001

criterion = nn.BCELoss()

optimizer = torch.optim.Adamax(model.parameters(), lr)


In [60]:
epochs = 10

counter = 0

print_every = 200

clip=5

if(train_on_gpu):
    model.cuda()


for e in range(epochs):
    # initialize hidden state
    h = model.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        model.zero_grad()

        # get the output from the model
        output, h = model(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            for inputs, labels in valid_loader:

                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            model.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/10... Step: 200... Loss: 0.328578... Val Loss: 0.483479
Epoch: 1/10... Step: 400... Loss: 0.483373... Val Loss: 0.491322
Epoch: 2/10... Step: 600... Loss: 0.254385... Val Loss: 0.613720
Epoch: 2/10... Step: 800... Loss: 0.280533... Val Loss: 0.429817
Epoch: 3/10... Step: 1000... Loss: 0.233427... Val Loss: 0.432853
Epoch: 3/10... Step: 1200... Loss: 0.256592... Val Loss: 0.452101
Epoch: 4/10... Step: 1400... Loss: 0.191377... Val Loss: 0.442684
Epoch: 4/10... Step: 1600... Loss: 0.213202... Val Loss: 0.446288
Epoch: 5/10... Step: 1800... Loss: 0.221942... Val Loss: 0.506222
Epoch: 5/10... Step: 2000... Loss: 0.249970... Val Loss: 0.449031
Epoch: 6/10... Step: 2200... Loss: 0.247803... Val Loss: 0.519646
Epoch: 6/10... Step: 2400... Loss: 0.087720... Val Loss: 0.553175
Epoch: 7/10... Step: 2600... Loss: 0.068131... Val Loss: 0.581983
Epoch: 7/10... Step: 2800... Loss: 0.052763... Val Loss: 0.545926
Epoch: 8/10... Step: 3000... Loss: 0.054882... Val Loss: 0.635725
Epoch: 8/10...

In [64]:
test_losses = []

num_correct = 0


h = model.init_hidden(batch_size)

model.eval()

for inputs, labels in test_loader:

  h = tuple([each.data for each in h])

  if(train_on_gpu):
    inputs, labels = inputs.cuda(), labels.cuda()

  output, h = model(inputs, h)
    
  # calculate loss
  test_loss = criterion(output.squeeze(), labels.float())
  test_losses.append(test_loss.item())
  
  # convert output probabilities to predicted class (0 or 1)
  pred = torch.round(output.squeeze())  # rounds to the nearest integer
  
  # compare predictions to true label
  correct_tensor = pred.eq(labels.float().view_as(pred))
  correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
  num_correct += np.sum(correct)

  # -- stats! -- ##
  # avg test loss
  print("Test loss: {:.3f}".format(np.mean(test_losses)))

  # accuracy over all test data
  test_acc = num_correct/len(test_loader.dataset)
  print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.878
Test accuracy: 0.016
Test loss: 0.983
Test accuracy: 0.032
Test loss: 0.864
Test accuracy: 0.049
Test loss: 0.776
Test accuracy: 0.065
Test loss: 0.793
Test accuracy: 0.082
Test loss: 0.791
Test accuracy: 0.097
Test loss: 0.716
Test accuracy: 0.116
Test loss: 0.676
Test accuracy: 0.133
Test loss: 0.667
Test accuracy: 0.150
Test loss: 0.655
Test accuracy: 0.167
Test loss: 0.630
Test accuracy: 0.185
Test loss: 0.671
Test accuracy: 0.200
Test loss: 0.686
Test accuracy: 0.214
Test loss: 0.680
Test accuracy: 0.231
Test loss: 0.690
Test accuracy: 0.247
Test loss: 0.691
Test accuracy: 0.263
Test loss: 0.681
Test accuracy: 0.280
Test loss: 0.669
Test accuracy: 0.298
Test loss: 0.671
Test accuracy: 0.314
Test loss: 0.666
Test accuracy: 0.331
Test loss: 0.662
Test accuracy: 0.348
Test loss: 0.666
Test accuracy: 0.363
Test loss: 0.664
Test accuracy: 0.381
Test loss: 0.677
Test accuracy: 0.396
Test loss: 0.684
Test accuracy: 0.412
Test loss: 0.675
Test accuracy: 0.430
Test loss: 0

In [66]:
review = 'this film lacked something i couldn  t put my finger on at first charisma on the part of the leading actress . this inevitably translated to lack of chemistry when she shared the screen with her leading man . even the romantic scenes came across as being merely the actors at play . it could very well have been the director who miscalculated what he needed from the actors . i just don  t know .  br    br   but could it have been the screenplay  just exactly who was the chef in love with  he seemed more enamored of his culinary skills and restaurant  and ultimately of himself and his youthful exploits  than of anybody or anything else . he never convinced me he was in love with the princess .  br    br   i was disappointed in this movie . but  don  t forget it was nominated for an oscar  so judge for yourself .'

In [68]:
def tokenize(review):

  review_test = review.lower()

  review_text = ''.join([c for c in review_test if c not in punctuation])

  words = review_text.split()

  test_ints = []

  test_ints.append([vocab_to_int[word] for word in words])

  return test_ints

In [84]:
def predict(net, test_review, sequence_length=200):
    
    net.eval()
    
    # transformando a review em tokens
    test_ints = tokenize(test_review)
    
    seq_length=sequence_length
  
    #Obtendo as features do tokens da reviews
    features = pad_features(test_ints, seq_length)
    
    # convertendo as features em tensor do pytorch
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # inicializando a rede neural
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # obtendo o output do modelo
    output, h = net(feature_tensor, h)
    
    pred = torch.round(output.squeeze()) 
    
    #retornando a resposta da predição
    if(pred.item()==1):
        print("Review positiva")
    else:
        print("Review negativa")

In [85]:
predict(model, review, 200)

Review negativa
